# Microsoft Teams Meetings Web Scrape

This script is used to scrape meetings data form the Microsoft Teams admin interface. We will use this data to create a graph representation of the the data (which is supplied below)

Currently, Microsoft Teams has no way to lookup and view attendees of meetings. This webscrape utilises Python selenium to scrape from an infinite scroll site to generate meeting data. 

The scrape captures: 
- meeting participants
- MeetingID
- Time of meeting

## 0. Import Packages

In [8]:
import urllib.request
from selenium.webdriver import ActionChains
import sys
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import re
from selenium import webdriver
import time
import csv
import pandas as pd
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException   
from selenium.webdriver.common.by import By
from selenium.webdriver.support.events import EventFiringWebDriver, AbstractEventListener
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [11]:
def check_exists_by_xpath(xpath, driver):     #Checks if a certain element exists when loading 
    try:
        driver.find_element_by_xpath(xpath)
        print('it exists')
    except NoSuchElementException:
        return False
    return True

## 1. User names from Microsoft (can get from admin.office)

In [32]:
def get_names():
    file = pd.read_csv(r"listofnames.csv") #change for VM
    return file['<DisplayName>'].tolist()

## 2. Build Scraper

In [33]:
def scraper(
    driver,
    name,
    search,
    df,
    people,
    f,
    ):
    writer = csv.writer(f)
    search.send_keys(name)
    try:
        WebDriverWait(driver,
                      25).until(EC.presence_of_element_located((By.PARTIAL_LINK_TEXT,
                                name)))

        # waits until a link with "Name" appears

        link = \
            driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[2]/main/div/div[2]/div[2]/div[2]/div[2]/div[1]/div[1]/div[2]/span/a'
                )
        try:
            link.click()
            WebDriverWait(driver,
                          60).until(EC.presence_of_element_located((By.XPATH,
                                    '/html/body/div[1]/div/div/div/div[2]/main/div/div[2]/div[2]/div/div/div/div[1]/ul/button[3]'
                                    )))

            try:
                call_history = \
                    driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[2]/main/div/div[2/div[2]/div/div/div/div[1]/ul/button[3]'
                        )  # students may not have any call history
                call_history.click()
            except NoSuchElementException:

                print 'try again'

            try:
                WebDriverWait(driver,
                              5).until(EC.visibility_of_all_elements_located((By.XPATH,
                        "//*[@class='link__apLink___2FU1J link__light___2m8u9']"
                        )))
                for i in range(0, 4):
                    check_exists_by_xpath("//*[@class='link__apLink___2FU1J link__light___2m8u9']"
                            , driver)
                    WebDriverWait(driver,
                                  60).until(EC.visibility_of_all_elements_located((By.XPATH,
                            "//*[@class='link__apLink___2FU1J link__light___2m8u9']"
                            )))

                    element = \
                        driver.find_elements_by_xpath("//*[@class='link__apLink___2FU1J link__light___2m8u9']"
                            )

                    for a in element:

                        url = a.get_attribute('href')
                        url_match = \
                            re.match(r"(.+)(users\/)(.+)(\/)(calls-session|meeting)(\/|\/\/)(.+)"
                                , url)
                        df = df.append({
                            'Name': name,
                            'User ID': url_match[3],
                            'Session ID': url_match[7],
                            'Time': a.get_attribute('text'),
                            }, ignore_index=True)
                        writer.writerow(df.iloc[-1, :])  # write lines of df to file

                    WebDriverWait(driver,
                                  60).until(EC.presence_of_element_located((By.XPATH,
                            "//*[@id='ap-AppMainContent']/div/div[2]/div[3]/div/div[2]/div[2]/div[1]/div[last()]"
                            )))  # looks for the last element

                    row_to_click = \
                        driver.find_element_by_xpath("//*[@id='ap-AppMainContent']/div/div[2]/div[3]/div/div[2]/div[2]/div[1]/div[last()]"
                            )
                    actions = ActionChains(driver)
                    actions.move_to_element(row_to_click).click().perform()  # highlights into the window
                    ActionChains(driver).send_keys(Keys.PAGE_DOWN
                            * 2).perform()  # sends page down instruction

                
                ActionChains(driver).send_keys(Keys.PAGE_UP
                        * 2).perform()

                WebDriverWait(driver,
                              60).until(EC.presence_of_element_located((By.XPATH,
                        '/html/body/div[1]/div/div/div/div[2]/main/div/div[1]/div[1]/div/div/ol/li[1]/a'
                        )))
                user_breadcrumb = \
                    driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[2]/main/div/div[1]/div[1]/div/div/ol/li[1]/a'
                        )
                user_breadcrumb.click()
                WebDriverWait(driver,
                              60).until(EC.presence_of_element_located((By.XPATH,
                        "//*[@class='ms-SearchBox-field SearchBox__field___accFh']"
                        )))
                search = \
                    driver.find_element_by_xpath("//*[@class='ms-SearchBox-field SearchBox__field___accFh']"
                        )
                print name
                print people.index(name)
                time.sleep(0.2)
                search.send_keys(Keys.CONTROL + 'a')
                search.send_keys(Keys.DELETE)
            except:

                WebDriverWait(driver,
                              60).until(EC.presence_of_element_located((By.XPATH,
                        '/html/body/div[1]/div/div/div/div[2]/main/div/div[1]/div[1]/div/div/ol/li[1]/a'
                        )))
                user_breadcrumb = \
                    driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[2]/main/div/div[1]/div[1]/div/div/ol/li[1]/a'
                        )
                user_breadcrumb.click()
                WebDriverWait(driver,
                              60).until(EC.presence_of_element_located((By.XPATH,
                        "//*[@class='ms-SearchBox-field SearchBox__field___accFh']"
                        )))
                search = \
                    driver.find_element_by_xpath("//*[@class='ms-SearchBox-field SearchBox__field___accFh']"
                        )
                print name
                print people.index(name)
                time.sleep(0.2)
                search.send_keys(Keys.CONTROL + 'a')
                search.send_keys(Keys.DELETE)
        except:
    except:

        search.send_keys(Keys.CONTROL + 'a')
        search.send_keys(Keys.DELETE)


SyntaxError: Missing parentheses in call to 'print'. Did you mean print('try again')? (<ipython-input-33-f37590017201>, line 35)

## 2. Build Main

In [14]:
def main(ind):  # accepts index of first name 
    options = webdriver.ChromeOptions() # webdriver for Chrome
    urlpage = 'https://admin.teams.microsoft.com/'
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--test-type')
    pswd = '<pwd>'
    driver = webdriver.Chrome(chrome_options=options,
                              executable_path=r"<path of Chromedriver>")
    page = urllib.request.urlopen(urlpage)
    driver.set_window_size(1124, 850)
    df = pd.DataFrame()  # blank dataframe to store rows of data
    people = get_names()


    f = open(r"meetingsfile.csv", 'a+', newline='') # open CSV to write rows of df

    writer = csv.writer(f)

    website = driver.get(urlpage)
    uname = driver.find_element_by_xpath("//*[@id='i0116']")  # element of uname window
    button = driver.find_element_by_xpath("//*[@id='idSIButton9']")  # login pages

    time.sleep(1)
    uname.send_keys('login')


    time.sleep(1)
    button.click()
    time.sleep(0.3)
    try:
        pword = driver.find_element_by_xpath("//*[@id='passwordInput']")  # login pages
        pword.send_keys(pswd)
    except NoSuchElementException:
        print 'No Such Element'

    try:
        button2 = driver.find_element_by_xpath("//*[@id='submitButton']"
                )
        button2.click()
    except NoSuchElementException:
        print 'No Such Element'

    try:
        button = driver.find_element_by_xpath("//*[@id='idSIButton9']")
        button.click()
    except:
        print 'No Such Element'

    WebDriverWait(driver,
                  60).until(EC.presence_of_element_located((By.XPATH,
                            "//*[@id='ap-app']/div/div/div/div[2]/div[1]/div/div/div[1]/a/div"
                            )))

    element = \
        driver.find_element_by_xpath("//*[@id='ap-app']/div/div/div/div[2]/div[1]/div/div/div[1]/a/div"
            )  # expand the side menu
    element.click()

    time.sleep(0.2)

    user = \
        driver.find_element_by_xpath("//*[@id='ap-app']/div/div/div/div[2]/div[1]/div/div/div[2]/nav/ul/li[2]/a"
            )
    user.click()  # click user
    WebDriverWait(driver,
                  60).until(EC.presence_of_element_located((By.XPATH,
                            '/html/body/div/div/div/div/div[2]/main/div/div[2]/div[2]/div[1]/div/div/div[2]/div[1]/input'
                            )))  # wait until searh bar appears
    WebDriverWait(driver,
                  60).until(EC.presence_of_element_located((By.XPATH,
                            '/html/body/div[1]/div/div/div/div[2]/main/div/div[2]/div[2]/div[2]/div[2]/div[1]/div[1]/div[2]/span/a'
                            )))  # wait until first link appears before inputting into search bar

        for name in people[ind:]:
        search = \
            driver.find_element_by_xpath('/html/body/div/div/div/div/div[2]/main/div/div[2]/div[2]/div[1]/div/div/div[2]/div[1]/input'
                )
        try:
            scraper(
                driver,
                name,
                search,
                df,
                people,
                f,
                )
        except Exception:
            print 'Time out (main)'
            main(people.index(name))  # if fails, run main again from last index of name


## Output example:

The output of the above script looks like the following

In [31]:
output = pd.read_csv(r"meetingsfile.csv")
columns_array = output.columns.values
output.rename(columns = {columns_array[1]:'ID1', columns_array[2]: 'Time', columns_array[3]: 'Met_with_ID2'}).iloc[:,1:4].head()

,ID1,Time,Met_with_ID2
0,2178e5ca-c396-4460-b747-35d2dc00a2c7,"Jun 17, 2020, 8:14 PM GMT+10",2b33b1a9-11e2-4ad7-9f19-c2c1d0e491d3
1,32b05a78-91ae-4dc5-a8e4-434660c7804c,"Jun 17, 2020, 8:04 PM GMT+10",2b33b1a9-11e2-4ad7-9f19-c2c1d0e491d3
2,b09069b4-977a-4cc0-87bc-f4770866af72,"Jun 17, 2020, 7:54 PM GMT+10",2b33b1a9-11e2-4ad7-9f19-c2c1d0e491d3
3,c1628bbf-0426-4ba9-ae47-470ee9e7f3ad,"Jun 17, 2020, 7:44 PM GMT+10",2b33b1a9-11e2-4ad7-9f19-c2c1d0e491d3
4,3e446d19-ab8f-4c01-9d77-83d90655e1bc,"Jun 17, 2020, 7:34 PM GMT+10",2b33b1a9-11e2-4ad7-9f19-c2c1d0e491d3
